In [2]:
# genetic_algorithm_continuous.py
from __future__ import annotations
import random
from typing import Callable, List, Tuple

# ----- Problem definition -----------------------------------------------------
# Maximize this objective (replace with your own)
def objective(xy: List[float]) -> float:
    x, y = xy
    return - (x - 1.0) ** 2 - (y + 2.0) ** 2 + 10.0

BOUNDS = [(-5.0, 5.0), (-5.0, 5.0)]  # per-gene (x, y) bounds

# ----- GA components ----------------------------------------------------------
Individual = List[float]

def init_individual(bounds: List[Tuple[float, float]]) -> Individual:
    return [random.uniform(lo, hi) for lo, hi in bounds]

def clip(ind: Individual, bounds: List[Tuple[float, float]]) -> Individual:
    return [max(lo, min(v, hi)) for v, (lo, hi) in zip(ind, bounds)]

def tournament_select(pop: List[Individual], fitness: List[float], k: int = 3) -> Individual:
    idxs = random.sample(range(len(pop)), k)
    best_idx = max(idxs, key=lambda i: fitness[i])
    return pop[best_idx][:]  # copy

def arithmetic_crossover(p1: Individual, p2: Individual, cx_prob: float = 0.9) -> Tuple[Individual, Individual]:
    if random.random() > cx_prob:
        return p1[:], p2[:]
    # BLX-like / arithmetic mix per gene with random alpha in [0,1]
    c1, c2 = [], []
    for a, b in zip(p1, p2):
        alpha = random.random()
        c1.append(alpha * a + (1 - alpha) * b)
        c2.append(alpha * b + (1 - alpha) * a)
    return c1, c2

def gaussian_mutation(ind: Individual, bounds: List[Tuple[float, float]], mut_prob: float = 0.1) -> Individual:
    # noise scale ~ 10% of range
    out = ind[:]
    for i, (lo, hi) in enumerate(bounds):
        if random.random() < mut_prob:
            sigma = 0.1 * (hi - lo)
            out[i] += random.gauss(0.0, sigma)
    return clip(out, bounds)

def evolve(
    objective_fn: Callable[[Individual], float],
    bounds: List[Tuple[float, float]],
    pop_size: int = 60,
    generations: int = 80,
    crossover_prob: float = 0.9,
    mutation_prob: float = 0.1,
    tournament_k: int = 3,
    elitism: int = 2,
    seed: int | None = 42,
):
    if seed is not None:
        random.seed(seed)

    # Initialize population
    population = [init_individual(bounds) for _ in range(pop_size)]
    fitness = [objective_fn(ind) for ind in population]

    for gen in range(generations):
        # Elitism: keep top-N
        elites = sorted(zip(population, fitness), key=lambda t: t[1], reverse=True)[:elitism]
        new_pop: List[Individual] = [e[0][:] for e in elites]

        # Generate offspring
        while len(new_pop) < pop_size:
            p1 = tournament_select(population, fitness, k=tournament_k)
            p2 = tournament_select(population, fitness, k=tournament_k)
            c1, c2 = arithmetic_crossover(p1, p2, crossover_prob)
            c1 = gaussian_mutation(c1, bounds, mutation_prob)
            if len(new_pop) < pop_size:
                new_pop.append(c1)
            c2 = gaussian_mutation(c2, bounds, mutation_prob)
            if len(new_pop) < pop_size:
                new_pop.append(c2)

        population = new_pop
        fitness = [objective_fn(ind) for ind in population]

        # Simple progress print
        if (gen + 1) % 10 == 0 or gen == 0 or gen == generations - 1:
            best_val = max(fitness)
            print(f"Gen {gen+1:03d} | Best fitness: {best_val:.6f}")

    # Return best individual and fitness
    best_idx = max(range(pop_size), key=lambda i: fitness[i])
    return population[best_idx], fitness[best_idx]


if __name__ == "__main__":
    best_sol, best_fit = evolve(
        objective_fn=objective,
        bounds=BOUNDS,
        pop_size=60,
        generations=80,
        crossover_prob=0.9,
        mutation_prob=0.1,
        tournament_k=3,
        elitism=2,
        seed=42,
    )
    print("\nBest solution:", best_sol)
    print("Best fitness :", best_fit)


Gen 001 | Best fitness: 9.875053
Gen 010 | Best fitness: 9.999999
Gen 020 | Best fitness: 10.000000
Gen 030 | Best fitness: 10.000000
Gen 040 | Best fitness: 10.000000
Gen 050 | Best fitness: 10.000000
Gen 060 | Best fitness: 10.000000
Gen 070 | Best fitness: 10.000000
Gen 080 | Best fitness: 10.000000

Best solution: [0.9999999848694612, -2.0000000046399165]
Best fitness : 10.0


Here we implement a genetic algorithm to find the maximum value of a given mathematical function (the objective function). It starts with a random population of possible solutions (individuals), then iteratively improves them over generations by selecting the best ones, combining their characteristics (crossover), and introducing small random changes (mutation). The process continues until a near-optimal solution is found, which is then returned along with its fitness value.